# Upload video stimuli to s3

In [24]:
#Which experiment? bucket_name is the name of the experiment and will be name of the databases both on mongoDB and S3
# bucket_name = 'human-physics-benchmarking-dominoes-pilot' #dominoes
bucket_name = 'human-physics-benchmarking-towers-pilot' #towers

In [25]:
import os
from glob import glob
import boto3
import botocore
from IPython.display import clear_output
import json
import pandas as pd
from PIL import Image

In [26]:
def list_files(paths, ext='mp4'):
    """Pass list of folders if there are stimuli in multiple folders. 
    Make sure that the containing folder is informative, as the rest of the path is ignored in naming. 
    Also returns filenames as uploaded to S3"""
    if type(paths) is not list:
        paths = [paths]
    results = []
    names = []
    for path in paths:
        results += [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        names += [os.path.basename(os.path.dirname(y))+'_'+os.path.split(y)[1] for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    return results,names

In [27]:
## helper to speed things up by not uploading images if they already exist, can be overriden 
def check_exists(s3, bucket_name, stim_name):
    try:
        s3.Object(bucket_name,stim_name).load()    
        return True
    except botocore.exceptions.ClientError as e:    
        if (e.response['Error']['Code'] == "404"):
            print('The object does not exist.')
            return False
        else:
            print('Something else has gone wrong with {}'.format(stim_name))

Pass list of folders if there are stimuli in multiple folders. Make sure that the containing folder is informative, as the rest of the path is ignored in naming.

In [28]:
## set up paths, etc.
# paths_to_stim = ['./example'] ## provide a list of full paths here.
# paths_to_stim = [
    
#     # 3 domino trials
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_SJ025_boxroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_SJ025_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_default_boxroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_SJ020_d3_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_SJ020_o1flex_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_SJ020_d3chairs_o1plants_tdwroom/",
    
#     # 0 domino trials
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom/",
    
#     # 1 domino trials
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_1mid_J025R45_o1flex_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_1mid_J025R45_boxroom/",
    
#     # 2 domino trials
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_2mid_J025R30_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom/",  
    
#     # "Catch trials" -- peoplen should be close to 100% 
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_4midRM1_tdwroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_4midRM1_boxroom/",
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_4mid_tdwroom/",    
#     "/Users/dbear/neuroailab/physics_benchmarking/stimuli/pilot_dominoes_4mid_boxroom/"
    
# ]

# paths_to_stim = glob.glob()
full_stim_paths, filenames = [x for x in list_files(paths_to_stim) if x !='.DS_Store'] #generate filenames and stimpaths
print('We have {} stimuli to upload.'.format(len(full_stim_paths)))    

We have 70 stimuli to upload.


In [51]:
local_stem = '/Users/dbear/neuroailab/physics_benchmarking/stimuli/'
dirnames = [d.split('/')[-1] for d in glob('./generation/pilot-towers/pilot_towers*')]
paths_to_stim = [local_stem + d for d in dirnames]

full_stim_paths, filenames = [x for x in list_files(paths_to_stim) if x !='.DS_Store'] #generate filenames and stimpaths
print('We have {} stimuli to upload.'.format(len(full_stim_paths)))    

We have 140 stimuli to upload.


In [48]:
# filenames

Upload to S3. This expects the `.aws/credentials` file in your home directory.

In [52]:
reallyRun = 1
if reallyRun:

    ## establish connection to s3 
    s3 = boto3.resource('s3')

    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=bucket_name) 
        print('Created new bucket.')
#     except NoCredentialsError:
#         print("Credential missing") #.aws/credentials should be in home folder, not in repo folder
    except:
        b = s3.Bucket(bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = False
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_stim_paths):        # use sorted(full_stim_paths) when not using photodraw32
        stim_name = filenames[i]
        if ((check_exists(s3, bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} as {} | {} of {}'.format(os.path.split(path_to_file)[-1],stim_name,(i+1),len(full_stim_paths)))
            s3.Object(bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(os.path.split(path_to_file)[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)
print('Done!')

Done!


In [53]:
for my_bucket_object in b.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0000_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0001_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0002_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0003_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0004_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0005_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarking-towers-pilot', key='pilot_towers_nb2_fr015_SJ010_mono0_tdwroom_0006_img.mp4')
s3.ObjectSummary(bucket_name='human-physics-benchmarkin